In [0]:
fileroot = "clinicaltrial_2021"

In [0]:
clinicaltrial_file = ("/FileStore/tables/"+fileroot+".csv")
pharma = ('dbfs:/FileStore/tables/pharma.csv')

In [0]:
dbutils.fs.ls("/FileStore/tables/"+fileroot+".csv")

In [0]:
dbutils.fs.ls("/FileStore/tables/")

In [0]:
dbutils.fs.head("/FileStore/tables/"+fileroot+".csv")

In [0]:
clinicaltrial_df = spark.read.csv("/FileStore/tables/"+fileroot+".csv",sep="|",header=True,inferSchema=True)

clinicaltrial_df.show(3, truncate=False)

In [0]:
#for line in dbutils.fs.head("/FileStore/tables/"+fileroot+".csv").splitlines():
#    print(line)

In [0]:
clinicaltrial_df.printSchema()


In [0]:
#The number of studies in the dataset.

clinicaltrial_df.distinct().count()

In [0]:
# list all the types (as contained in the Type column) of studies in thedataset along with the frequencies of each type.

In [0]:
from pyspark.sql.functions import desc,explode,count,split,col

In [0]:
studiestype_df = clinicaltrial_df.groupBy("type").count().orderBy(desc("count"))

studiestype_df.show()

In [0]:
# top 5 conditions (from Conditions) with their frequencies.

In [0]:
conditions_df = clinicaltrial_df.where(clinicaltrial_df["conditions"].isNotNull())\
.select(explode(split("conditions",",")).alias("top_5_conditions"))

In [0]:
freq_df = conditions_df.groupBy("top_5_conditions").count().orderBy(desc("count"))
freq_df.show(5)

In [0]:
#Find the 10 most common sponsors that are not pharmaceutical companies, alongwith the number of clinical trials they have sponsored. 

In [0]:
pharma_df = spark.read.csv(pharma,header=True,inferSchema=True,sep=",")

pharma_df.show(2, truncate=False)

In [0]:
#grouping pharma dataset by count
pharma_df.groupBy("Parent_Company").count().show(2)

In [0]:
#grouping clinical_trial dataset by count
clinicaltrial_df.groupBy("Sponsor").count().show(2)

In [0]:
#joining sponsors data with parent companies that are not pharmaceutical companies
jo = clinicaltrial_df.join(pharma_df,clinicaltrial_df.Sponsor==pharma_df.Parent_Company,"left")
jo.show(2)

In [0]:
spnsr = jo.filter(jo.Parent_Company.isNull())

spnsr.show(1)

In [0]:
spnsr_f = spnsr.distinct().groupBy('Sponsor').count().orderBy('count',ascending=False)

spnsr_f.show(10,truncate=False)

In [0]:
#Plot number of completed studies each month in a given year – for the submission dataset, the year is 2021. You need to include your visualization as well as a table of all the values you have plotted for each month

In [0]:
import pyspark.sql.functions as jk 

In [0]:
clinicaltrial_df.select("Status", "Completion").na.drop()\
.filter(jk.col("Status") == "Completed")\
.withColumn("Month", jk.split("Completion"," ").getItem(0))\
.withColumn("Year", jk.split("Completion"," ").getItem(1))\
.filter(jk.col("Year") == "2021")\
.drop("Status","Completion", "Year")\
.groupBy("Month")\
.count()\
.show()